In [1]:
from et_lib.ET_Data_Reader import DatabaseHistoricalData
from datetime import datetime

startDate = datetime.strptime("12-31-2010","%m-%d-%Y")
endDate   = datetime.today()
FIA=["G5 Allocation Equities FIC FIA","VIC RV FIC FIA","Honor Master FIA"]
FIM=["G5 Allocation FIC FIM","VIC Mult FIC FIM","Ares Mult FIC FIM"]
BM={"IBX":FIA,"IFMM":FIM}

db=DatabaseHistoricalData(startDate,endDate,FIM+FIA+list(BM.keys()))
df=db.getData().dropna()
rets=(df/df.shift(1)-1).dropna()
print(df.index)

DatetimeIndex(['2013-05-24', '2013-05-27', '2013-05-28', '2013-05-29',
               '2013-05-31', '2013-06-03', '2013-06-04', '2013-06-05',
               '2013-06-06', '2013-06-07',
               ...
               '2022-11-21', '2022-11-22', '2022-11-23', '2022-11-24',
               '2022-11-25', '2022-11-28', '2022-11-29', '2022-11-30',
               '2022-12-01', '2022-12-02'],
              dtype='datetime64[ns]', length=2356, freq=None)


In [40]:
from sklearn.linear_model import LinearRegression
import numpy as np


text={}
for b in BM:
    text[b]={}
    X=rets[b].values
    for f in BM[b]:
        y = rets[f].values
        reg=LinearRegression().fit(X,y)
        TE=((((X-y)**2).sum()/(len(X)-1))*252)**0.5                                                                             #type: ignore
        print(f+": "+str(round(reg.coef_[0][0],2))+"*"+b+" "+str(round(reg.intercept_[0]*252,2))+"  / TE: "+str(round(TE,2)))   #type: ignore
        text[b][f]={"reg":str(round(reg.coef_[0][0],2))+"*"+b+" "+str(round(reg.intercept_[0]*252,2)),"TE":str(round(TE,2))}    #type: ignore
        

G5 Allocation Equities FIC FIA: 0.76*IBX 0.01  / TE: 0.11
VIC RV FIC FIA: 0.76*IBX 0.02  / TE: 0.1
Honor Master FIA: 0.71*IBX 0.03  / TE: 0.12
G5 Allocation FIC FIM: 0.83*IFMM 0.02  / TE: 0.02
VIC Mult FIC FIM: 1.18*IFMM -0.0  / TE: 0.03
Ares Mult FIC FIM: 1.17*IFMM -0.01  / TE: 0.03


In [92]:
import plotly.express as px
import pandas as pd

for b in BM:
    bm=rets[b]
    bm.index.rename("Data",inplace=True)
    bm.columns=[b]
    X=rets[[b]+BM[b]]
    X=X.droplevel(1,axis=1)
    X=X.stack()
    X.index.rename(["Data","Ativo"],inplace=True)
    X=pd.DataFrame(X,columns=["fundo"])
    X=X.join(bm)
    X["Ativo"]=[i[1] for i in X.index]
    X=X.loc[[i[1]!=b for i in X.index]]

    fig=px.scatter(X,x=b,y="fundo",facet_col="Ativo", trendline="ols")
    
    fig.show()
    

In [6]:
import plotly.express as px

for b in BM:
    data=rets[[b]+BM[b]]
    data=data.droplevel(1,axis=1)
    fig=px.imshow(data.corr())
    fig.show()

